In [1]:
# 6d239b340893465b9acdd0484b7417f6 for stock news newsapi
# 7AW5JZn1DOHDAyYN8idPP2N8N7SA1LGN for apilayer

In [8]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime

current_date = datetime.now().strftime("%Y-%m-%d")  # Get the current date in "YYYY-MM-DD" format
finviz_url = 'https://finviz.com/quote.ashx?t='
tickers = ['GOOG']

news_tables = {}
for ticker in tickers:
    url = finviz_url + ticker

    req = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(req)

    html = BeautifulSoup(response, features='html.parser')

    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    break

key = next(iter(news_tables))
stock_data = news_tables[key]
stock_rows = stock_data.findAll('tr')
parsed_data = []
prev_date = None
for index, row in enumerate(stock_rows):
    title_ = row.a
    if title_:
        title = title_.text
    timestamp_ = row.td
    if timestamp_:
        timestamp = timestamp_.text
        date_data = [data.strip() for data in timestamp.split(" ") if data.strip()]  # Remove empty strings
        # print(date_data)

        if "Today" in date_data:
            date_data[date_data.index("Today")] = current_date
            prev_date =  current_date

        if len(date_data) == 1:
            time = date_data[0]
            date = prev_date
        else:
            date = date_data[0]
            time = date_data[1]
    parsed_data.append([key,date,time,title])

df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])
vader = SentimentIntensityAnalyzer()
df['compound'] = df['title'].apply(lambda title: vader.polarity_scores(title)['compound'] if title else 0)
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date

df

,ticker,date,time,title,compound
0,GOOG,2023-09-23,09:44AM,30 Low Cost Business Ideas With High Profit,0.2023
1,GOOG,2023-09-23,07:35AM,Google's Startling Admission Confirms What Inv...,-0.1531
2,GOOG,2023-09-23,06:14AM,25 Most Mentally Stimulating Jobs According to...,0.4902
3,GOOG,2023-09-23,05:50AM,Which Top Artificial Intelligence (AI) Stock C...,0.5994
4,GOOG,2023-09-23,05:31AM,25 High Paying Jobs That Dont Require Math,0.0000
...,...,...,...,...,...
98,GOOG,2023-09-23,10:14AM,Google wants part of antitrust trial kept out ...,0.0000
99,GOOG,2023-09-23,09:56AM,Microsoft Eyes Apples Spot as Worlds Largest S...,0.0000
100,GOOG,2023-09-23,08:07AM,Dont Fret the Fed. Buy Meta and These Stocks a...,0.0000
101,GOOG,2023-09-23,06:38AM,10 Best Ad-tech Stocks To Buy Now,0.6369


In [6]:
df["date"].unique

<bound method Series.unique of 0      2023-09-23
1      2023-09-23
2      2023-09-23
3      2023-09-23
4      2023-09-23
          ...    
98            NaT
99            NaT
100           NaT
101           NaT
102           NaT
Name: date, Length: 103, dtype: object>